In [23]:
import mlagents
import random
import numpy as np
from mlagents_envs.environment import UnityEnvironment as UE

In [24]:
env = UE(file_name="4444_project",seed=1,side_channels=[])
env.reset()

In [25]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['CarBehavior?team=0']
CarBehavior?team=0


In [26]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')], action_spec=ActionSpec(continuous_size=1, discrete_branches=()))


In [27]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')


In [28]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 1, Discrete: ()


In [29]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [30]:
print(decision_steps)
print(list(decision_steps))

[0]


In [31]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [32]:
action = spec.action_spec.random_action(len(decision_steps))
print(action)
print(type(action.discrete))
print(action.discrete)
# action = action.continuous

<class 'numpy.ndarray'>
[]


In [33]:
env.set_actions(behavior_name, action)

In [ ]:
for episode in range(10):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        # Generate an action for all agents
        action = spec.action_spec.random_action(len(decision_steps))
        #action = spec.action_spec.empty_action(len(decision_steps))
        #action.add_continuous(np.array([1]))
        print(action.continuous)
        #action.add_continuous(np.array([[0]]))
        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        #print(decision_steps.obs)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

[[-0.03131628]]
[[-0.90906036]]
[[-0.5950484]]
[[0.234864]]
[[0.12898035]]
[[0.52679944]]
[[-0.88467735]]
[[-0.59727436]]
[[0.13126513]]
[[-0.03948199]]
[[-0.9299149]]
[[-0.909936]]
[[0.7430725]]
[[0.7051913]]
[[-0.91952616]]
[[-0.3471991]]
[[0.6351867]]
[[0.34568062]]
[[-0.94105494]]
[[-0.87808466]]
[[0.04906436]]
[[0.6455883]]
[[0.41399238]]
[[0.06046845]]
[[0.47060615]]
[[-0.6723671]]
[[0.34451893]]
[[-0.38909182]]
[[-0.9209895]]
[[0.7508235]]
[[-0.8858137]]
[[-0.86763227]]
[[0.6908816]]
[[-0.91107816]]
[[-0.97089505]]
[[0.09096988]]
[[0.07896225]]
[[0.4318977]]
[[0.9490556]]
[[-0.44427064]]
[[-0.12782106]]
[[-0.8678273]]
[[-0.63040763]]
[[-0.9178301]]
[[0.894039]]
[[-0.1119526]]
[[-0.5567822]]
[[-0.73592377]]
[[-0.39068252]]
[[-0.13249046]]
[[-0.4233097]]
[[-0.86308986]]
[[-0.8842528]]
[[-0.13482137]]
[[-0.76154506]]
[[-0.80016005]]
[[-0.25482616]]
[[0.9302672]]
[[-0.10953804]]
[[-0.8391985]]
[[0.4844356]]
[[-0.44186732]]
[[-0.61236113]]
[[-0.92425853]]
[[0.15305899]]
[[-0.04267386

[[0.31612164]]
[[-0.2558609]]
[[0.5326418]]
[[0.77495587]]
[[0.3382529]]
[[0.5960491]]
[[-0.7274234]]
[[0.47128215]]
[[0.6491123]]
[[-0.78860193]]
[[-0.06596246]]
[[0.6583345]]
[[0.03122222]]
[[0.55280364]]
[[0.00803827]]
[[-0.54552364]]
[[0.7515887]]
[[0.22706005]]
[[0.69138736]]
[[-0.01668016]]
[[0.15858816]]
[[-0.9292366]]
[[-0.7636693]]
[[-0.36797825]]
[[0.13972667]]
[[-0.4896286]]
[[-0.7928375]]
[[0.21472138]]
[[0.5439087]]
[[0.4942074]]
[[0.6961867]]
[[0.87252855]]
[[-0.13245326]]
[[-0.61831665]]
[[-0.6086916]]
[[-0.78850555]]
[[0.16632885]]
[[0.52944237]]
[[0.3287163]]
[[0.37166885]]
[[0.8513415]]
[[-0.43656993]]
[[0.04959744]]
[[0.9960505]]
[[0.7084609]]
[[0.8111389]]
[[-0.21340862]]
[[-0.03888648]]
[[0.5980245]]
[[-0.7544823]]
[[0.20621873]]
[[0.74342334]]
[[-0.06338815]]
[[0.00781033]]
[[0.5504491]]
[[-0.91275513]]
[[-0.9091115]]
[[0.74109143]]
[[0.62990326]]
[[0.4207385]]
[[0.42220098]]
[[-0.49221942]]
[[0.8659568]]
[[0.9884474]]
[[-0.54945856]]
[[-0.49591193]]
[[0.44448364]

In [13]:
env.close()